In [1]:
import mlflow

# Connect to Databricks workspace with submitted credentials or use stored credentials
mlflow.login()
# Set tracking URI to Databricks -  tell MLflow to send the data into Databricks Workspace
mlflow.set_tracking_uri("databricks")

2025/11/04 02:39:15 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-7d1169bb-4536.cloud.databricks.com.


In [2]:
# Timeouts on dependency collection even when given  20 minutes timeout with MLFLOW_REQUIREMENTS_INFERENCE_TIMEOUT
!poetry env activate
import os
import logging
import mlflow
from datetime import datetime
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO)
logging.getLogger("mlflow.utils.environment").setLevel(logging.DEBUG)


load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/agent-no-memory-{datetime.now()}"
try:
    mlflow.create_experiment(experiment_name)
except Exception as e:
    print(f"Experiment {experiment_name} already exists.")
mlflow.set_experiment(experiment_name)

mlflow.end_run()  # End any existing runs if they exist
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model="agent.py",
        name="Helpful agent without memory",
        input_example={
            "input": [
                {"role": "user", "content": "What is the full name of poet Byron?"}
            ]
        },  # Example input instead of default sample "Hello!"
        registered_model_name="workspace.default.helpful-agent-no-memory",
        pip_requirements="requirements.txt"  # Specify dependencies manually
    )
mlflow.end_run() # End the run after logging the model

source /mnt/e/Projekti/ml/databricks-timeseries/.venv/bin/activate


INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:code_model_950a1fbba0f64d34b8efd894fd65dfba:Initializing ToolCallingAgentNoMemory with model: gpt-5-nano
2025/11/04 02:39:33 INFO mlflow.pyfunc: Predicting on input example to validate output
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:code_mod

🏃 View run persistent-lynx-912 at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/654634460709250/runs/f562e2712b7749fcad421a6afc9ad69e
🧪 View experiment at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/654634460709250


In [ ]:
# Test the downloaded model/agent - useful for model prediction debugging; model is downloaded from databricks
# models:/ is a special MLflow URI scheme that refers to a model version registered in the MLflow Model Registry

import mlflow

mlflow.models.predict(
    model_uri="models:/m-763cdcbde5584da5b7caa2f47dcd960a",
    input_data={
        "input": [{"role": "user", "content": "What is the full name of poet Byron?"}]
    },
)

In [ ]:
# Save the model locally, useful to test collection of dependencies and test predictions locally
import mlflow
import shutil
import logging
import os
from datetime import datetime
from dotenv import load_dotenv

path = "./models/helpful-agent-no-memory-model"
shutil.rmtree(path, ignore_errors=True)

logging.basicConfig(level=logging.INFO)
logging.getLogger("mlflow.utils.environment").setLevel(logging.DEBUG)


load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/agent-no-memory-{datetime.now()}"
try:
    mlflow.create_experiment(experiment_name)
except Exception as e:
    print(f"Experiment {experiment_name} already exists.")
mlflow.set_experiment(experiment_name)

mlflow.end_run()  # End any existing runs if they exist
with mlflow.start_run():
    mlflow.pyfunc.save_model(
        path=path,
        python_model="agent.py",
    )
    # Upload model
    mlflow.register_model(
        model_uri="file:/mnt/e/Projekti/ml/databricks-timeseries/models/helpful-agent-no-memory-model",
        name="workspace.default.helpful-agent-no-memory"
    )
mlflow.end_run()  # End the run after logging the model